# The Case for Tail Hedging: Testing Spitznagel's Thesis

Spitznagel argues that a small allocation to deep OTM puts **improves geometric compounding** even though puts have negative expected value. The key:

$$G \approx \mu - \frac{\sigma^2}{2}$$

If puts reduce $\sigma^2$ enough, the variance drain savings exceed the premium cost.

**Critically**, Spitznagel's strategy uses **leverage**: 100% equity exposure + puts on top (not 97% equity + 3% puts). This is what we test here using the backtester's budget callable.

All results below use **real SPY options data (2008–2025)**.

In [ ]:
import os, sys, warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

PROJECT_ROOT = os.path.realpath(os.path.join(os.getcwd(), '..'))
sys.path.insert(0, PROJECT_ROOT)
sys.path.insert(0, os.path.join(PROJECT_ROOT, 'notebooks'))
os.chdir(PROJECT_ROOT)

from backtest_runner import (
    load_data, run_backtest, INITIAL_CAPITAL,
    make_puts_strategy, make_deep_otm_put_strategy,
)
from backtester.enums import Order
from nb_style import apply_style, shade_crashes, color_excess, style_returns_table, FT_GREEN, FT_RED, FT_BLUE

apply_style()
%matplotlib inline
print('Ready.')

In [ ]:
data = load_data()
schema = data['schema']
spy_prices = data['spy_prices']
years = data['years']

---
## 1. Variance Drain on Actual SPY

How much does volatility cost in compounding terms?

In [ ]:
daily_returns = spy_prices.pct_change().dropna()
arith_annual = daily_returns.mean() * 252
geom_annual = (1 + daily_returns).prod() ** (252 / len(daily_returns)) - 1
vol_annual = daily_returns.std() * np.sqrt(252)
drain = arith_annual - geom_annual

rolling_vol = daily_returns.rolling(252).std() * np.sqrt(252)
rolling_drain = (rolling_vol ** 2) / 2

fig, axes = plt.subplots(2, 1, figsize=(16, 8), sharex=True)

ax = axes[0]
ax.plot(rolling_vol.index, rolling_vol * 100, color=FT_BLUE, lw=1.2)
ax.set_ylabel('Annualized Vol (%)')
ax.set_title('SPY Rolling 1-Year Volatility', fontweight='bold')
shade_crashes(ax)
ax.legend(loc='upper right', fontsize=7)

ax = axes[1]
ax.fill_between(rolling_drain.index, rolling_drain * 100, 0, color=FT_RED, alpha=0.4)
ax.set_ylabel('Variance Drain (%/yr)')
ax.set_title('Rolling Variance Drain (σ²/2) — What Volatility Costs You in Compounding', fontweight='bold')
shade_crashes(ax)

plt.tight_layout()
plt.show()

print(f'Arithmetic mean: {arith_annual*100:.2f}%  |  Geometric mean: {geom_annual*100:.2f}%  |  Vol: {vol_annual*100:.1f}%')
print(f'Variance drain: {drain*100:.2f}%/yr  (theoretical σ²/2 = {(vol_annual**2/2)*100:.2f}%)')
print(f'Peak drain during GFC: {rolling_drain.max()*100:.1f}%/yr')

---
## 2. The AQR Test: No Leverage (Allocation Split)

This is what AQR tests and what always fails: reduce equity to fund puts. Of course this loses — you're reducing your best asset to buy an expensive hedge.

In [ ]:
no_leverage_configs = [
    ('SPY only',            1.0,    0.0,    lambda: make_deep_otm_put_strategy(schema)),
    ('Deep OTM 0.1%',      0.999,  0.001,  lambda: make_deep_otm_put_strategy(schema)),
    ('Deep OTM 0.5%',      0.995,  0.005,  lambda: make_deep_otm_put_strategy(schema)),
    ('Deep OTM 1.0%',      0.99,   0.01,   lambda: make_deep_otm_put_strategy(schema)),
    ('Deep OTM 3.3%',      0.967,  0.033,  lambda: make_deep_otm_put_strategy(schema)),
    ('Std OTM 1.0%',       0.99,   0.01,   lambda: make_puts_strategy(schema)),
]

no_lev_results = []
for name, s, o, fn in no_leverage_configs:
    print(f'  {name}...', end=' ', flush=True)
    r = run_backtest(name, s, o, fn, data)
    no_lev_results.append(r)
    print(f'annual {r["annual_ret"]:+.2f}%, excess {r["excess_annual"]:+.2f}%, DD {r["max_dd"]:.1f}%')

In [ ]:
rows = []
for r in no_lev_results:
    d = r['balance']['% change'].dropna()
    vol = d.std() * np.sqrt(252) * 100
    rows.append({
        'Strategy': r['name'], 'Annual %': r['annual_ret'],
        'Vol %': vol, 'Max DD %': r['max_dd'],
        'Excess %': r['excess_annual'], 'Trades': r['trades'],
    })
df = pd.DataFrame(rows)
styled = df.style.format({'Annual %': '{:.2f}', 'Vol %': '{:.1f}', 'Max DD %': '{:.1f}',
                           'Excess %': '{:+.2f}', 'Trades': '{:.0f}'}).map(color_excess, subset=['Excess %'])
style_returns_table(styled).set_caption('AQR framing: reduce equity to fund puts (NO leverage) — always loses')

AQR is right *in this framing*: every put allocation underperforms SPY.

But **this is not what Spitznagel proposes**.

---
## 3. The Spitznagel Test: With Leverage (100% Equity + Puts on Top)

Spitznagel's actual claim: keep **100% in equity** and add puts on top via a small budget. This is leverage — total exposure exceeds 100%. The backtester's `options_budget` callable does exactly this.

In [ ]:
# Leveraged: 100% SPY + puts funded by budget callable (exposure > 100%)
leverage_configs = [
    ('100% SPY (baseline)',    None),
    ('+ 0.05% deep OTM puts', 0.0005),
    ('+ 0.1% deep OTM puts',  0.001),
    ('+ 0.2% deep OTM puts',  0.002),
    ('+ 0.5% deep OTM puts',  0.005),
    ('+ 1.0% deep OTM puts',  0.01),
    ('+ 2.0% deep OTM puts',  0.02),
    ('+ 3.3% deep OTM puts',  0.033),
]

lev_results = []
for name, budget_pct in leverage_configs:
    print(f'  {name}...', end=' ', flush=True)
    bfn = None
    if budget_pct is not None:
        _bp = budget_pct
        bfn = lambda date, tc, bp=_bp: tc * bp
    r = run_backtest(name, 1.0, 0.0, lambda: make_deep_otm_put_strategy(schema), data, budget_fn=bfn)
    lev_results.append(r)
    print(f'annual {r["annual_ret"]:+.2f}%, excess {r["excess_annual"]:+.2f}%, DD {r["max_dd"]:.1f}%')

In [ ]:
rows = []
for r in lev_results:
    d = r['balance']['% change'].dropna()
    vol = d.std() * np.sqrt(252) * 100
    rows.append({
        'Strategy': r['name'], 'Annual %': r['annual_ret'],
        'Vol %': vol, 'Max DD %': r['max_dd'],
        'Excess %': r['excess_annual'], 'Trades': r['trades'],
    })
df_lev = pd.DataFrame(rows)
styled = df_lev.style.format({'Annual %': '{:.2f}', 'Vol %': '{:.1f}', 'Max DD %': '{:.1f}',
                               'Excess %': '{:+.2f}', 'Trades': '{:.0f}'}).map(color_excess, subset=['Excess %'])
style_returns_table(styled).set_caption('Spitznagel framing: 100% SPY + puts on top (WITH leverage)')

---
## 4. Capital Curves: AQR Framing vs Spitznagel Framing

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(18, 7))
spy_norm = spy_prices / spy_prices.iloc[0] * INITIAL_CAPITAL

# AQR framing (no leverage)
ax = axes[0]
ax.plot(spy_norm.index, spy_norm.values, 'k--', lw=2.5, label='SPY B&H', alpha=0.7)
cmap = plt.cm.Reds(np.linspace(0.3, 0.9, len(no_lev_results) - 1))
for r, c in zip(no_lev_results[1:], cmap):
    r['balance']['total capital'].plot(ax=ax, label=f"{r['name']} ({r['excess_annual']:+.2f}%)",
                                       color=c, alpha=0.8, lw=1.2)
shade_crashes(ax)
ax.set_title('AQR framing: Reduce equity to fund puts', fontweight='bold', color=FT_RED)
ax.set_ylabel('$')
ax.ticklabel_format(style='plain', axis='y')
ax.legend(fontsize=6, loc='upper left')

# Spitznagel framing (leverage)
ax = axes[1]
ax.plot(spy_norm.index, spy_norm.values, 'k--', lw=2.5, label='SPY B&H', alpha=0.7)
cmap = plt.cm.Greens(np.linspace(0.3, 0.9, len(lev_results) - 1))
for r, c in zip(lev_results[1:], cmap):
    r['balance']['total capital'].plot(ax=ax, label=f"{r['name']} ({r['excess_annual']:+.2f}%)",
                                       color=c, alpha=0.8, lw=1.2)
shade_crashes(ax)
ax.set_title('Spitznagel framing: 100% equity + puts on top', fontweight='bold', color=FT_GREEN)
ax.set_ylabel('$')
ax.ticklabel_format(style='plain', axis='y')
ax.legend(fontsize=6, loc='upper left')

plt.tight_layout()
plt.show()

---
## 5. Also Try With Standard OTM Puts (Leveraged)

Compare deep OTM (δ -0.10 to -0.02) vs standard OTM (δ -0.25 to -0.10) in the leveraged framing.

In [ ]:
std_lev_configs = [
    ('+ 0.1% std OTM puts',  0.001),
    ('+ 0.5% std OTM puts',  0.005),
    ('+ 1.0% std OTM puts',  0.01),
]

std_lev_results = [lev_results[0]]  # baseline
for name, budget_pct in std_lev_configs:
    print(f'  {name}...', end=' ', flush=True)
    _bp = budget_pct
    bfn = lambda date, tc, bp=_bp: tc * bp
    r = run_backtest(name, 1.0, 0.0, lambda: make_puts_strategy(schema), data, budget_fn=bfn)
    std_lev_results.append(r)
    print(f'annual {r["annual_ret"]:+.2f}%, excess {r["excess_annual"]:+.2f}%, DD {r["max_dd"]:.1f}%')

In [ ]:
# Side by side: deep OTM vs standard OTM, both leveraged
fig, axes = plt.subplots(1, 2, figsize=(18, 7))

for ax, group, title, palette in [
    (axes[0], lev_results, 'Deep OTM puts (δ -0.10 to -0.02) + 100% SPY', plt.cm.Purples),
    (axes[1], std_lev_results, 'Standard OTM puts (δ -0.25 to -0.10) + 100% SPY', plt.cm.Blues),
]:
    ax.plot(spy_norm.index, spy_norm.values, 'k--', lw=2.5, label='SPY B&H', alpha=0.7)
    cmap = palette(np.linspace(0.3, 0.9, max(len(group) - 1, 1)))
    for r, c in zip(group[1:], cmap):
        r['balance']['total capital'].plot(ax=ax, label=f"{r['name']} ({r['excess_annual']:+.2f}%)",
                                           color=c, alpha=0.85, lw=1.5)
    shade_crashes(ax)
    ax.set_title(title, fontweight='bold', fontsize=11)
    ax.set_ylabel('$')
    ax.ticklabel_format(style='plain', axis='y')
    ax.legend(fontsize=6, loc='upper left')

plt.tight_layout()
plt.show()

---
## 6. Crash-Period Trade Analysis

How much did puts actually pay during each crash? This is the make-or-break for the Spitznagel thesis.

In [ ]:
# Run the deep OTM 0.5% leveraged config and analyze its trades
r_analysis = run_backtest('Deep OTM 0.5% (leveraged)', 1.0, 0.0,
                          lambda: make_deep_otm_put_strategy(schema), data,
                          budget_fn=lambda date, tc: tc * 0.005)
trade_log = r_analysis['trade_log']

if len(trade_log) > 0:
    first_leg = trade_log.columns.levels[0][0]
    entry_mask = trade_log[first_leg]['order'].isin([Order.BTO, Order.STO])
    entries = trade_log[entry_mask]
    exits = trade_log[~entry_mask]

    trades = []
    for _, entry_row in entries.iterrows():
        contract = entry_row[first_leg]['contract']
        exit_rows = exits[exits[first_leg]['contract'] == contract]
        if exit_rows.empty:
            continue
        exit_row = exit_rows.iloc[0]
        entry_cost = entry_row['totals']['cost'] * entry_row['totals']['qty']
        exit_cost = exit_row['totals']['cost'] * exit_row['totals']['qty']
        pnl = -(entry_cost + exit_cost)
        trades.append({
            'entry_date': entry_row['totals']['date'],
            'exit_date': exit_row['totals']['date'],
            'pnl': pnl,
            'entry_cost': entry_cost,
        })

    trades_df = pd.DataFrame(trades)
    trades_df['entry_date'] = pd.to_datetime(trades_df['entry_date'])
    trades_df['exit_date'] = pd.to_datetime(trades_df['exit_date'])

    CRASHES = [
        ('2008 GFC',   '2007-10-01', '2009-06-01'),
        ('2020 COVID', '2020-01-01', '2020-06-01'),
        ('2022 Bear',  '2022-01-01', '2022-12-31'),
        ('Calm periods', None, None),
    ]

    def classify(d):
        for name, s, e in CRASHES[:-1]:
            if pd.Timestamp(s) <= d <= pd.Timestamp(e):
                return name
        return 'Calm periods'

    trades_df['period'] = trades_df['entry_date'].apply(classify)

    period_stats = trades_df.groupby('period').agg(
        trades=('pnl', 'count'),
        total_pnl=('pnl', 'sum'),
        avg_pnl=('pnl', 'mean'),
        best_trade=('pnl', 'max'),
        total_premium=('entry_cost', 'sum'),
    ).round(0)

    styled = period_stats.style.format({
        'total_pnl': '${:,.0f}', 'avg_pnl': '${:,.0f}',
        'best_trade': '${:,.0f}', 'total_premium': '${:,.0f}', 'trades': '{:.0f}'
    })
    style_returns_table(styled).set_caption('Deep OTM Put Trades: P&L by Market Period')
else:
    print('No trades executed.')

In [ ]:
if len(trades_df) > 0:
    # Per-trade P&L bar chart colored by period
    sorted_trades = trades_df.sort_values('entry_date')
    period_colors = {'2008 GFC': FT_RED, '2020 COVID': '#FF8833', '2022 Bear': '#9467bd', 'Calm periods': '#AAAAAA'}
    colors = [period_colors.get(p, '#999') for p in sorted_trades['period']]

    fig, axes = plt.subplots(2, 1, figsize=(16, 10))

    # Per-trade P&L
    ax = axes[0]
    ax.bar(range(len(sorted_trades)), sorted_trades['pnl'], color=colors, width=1.0, edgecolor='none')
    ax.axhline(0, color='black', lw=0.8)
    ax.set_xlabel('Trade #')
    ax.set_ylabel('P&L ($)')
    ax.set_title('Per-Trade P&L: Deep OTM Puts (colored by market period)', fontweight='bold')

    import matplotlib.patches as mpatches
    handles = [mpatches.Patch(color=c, label=l) for l, c in period_colors.items()]
    ax.legend(handles=handles, fontsize=8)

    # Cumulative P&L
    ax = axes[1]
    cum_pnl = sorted_trades.set_index('exit_date')['pnl'].cumsum()
    ax.plot(cum_pnl.index, cum_pnl.values, color=FT_BLUE, lw=2)
    ax.fill_between(cum_pnl.index, cum_pnl.values, 0,
                    where=cum_pnl.values >= 0, color=FT_GREEN, alpha=0.15)
    ax.fill_between(cum_pnl.index, cum_pnl.values, 0,
                    where=cum_pnl.values < 0, color=FT_RED, alpha=0.15)
    shade_crashes(ax)
    ax.axhline(0, color='black', lw=0.8, ls='--')
    ax.set_ylabel('Cumulative P&L ($)')
    ax.set_title('Cumulative Options P&L Over Time', fontweight='bold')
    ax.legend(loc='upper left', fontsize=7)

    plt.tight_layout()
    plt.show()

    total_premium = trades_df['entry_cost'].sum()
    total_pnl = trades_df['pnl'].sum()
    crash_pnl = trades_df[trades_df['period'] != 'Calm periods']['pnl'].sum()
    calm_pnl = trades_df[trades_df['period'] == 'Calm periods']['pnl'].sum()

    print(f'Total premium spent: ${abs(total_premium):,.0f}')
    print(f'Total P&L: ${total_pnl:,.0f}')
    print(f'Crash period P&L: ${crash_pnl:,.0f}')
    print(f'Calm period P&L: ${calm_pnl:,.0f}')
    print(f'Crash payoff / Total premium: {crash_pnl / abs(total_premium) * 100:.1f}%')

---
## 7. Drawdown During Crashes: Does the Hedge Actually Reduce Max DD?

In [ ]:
# Compare drawdowns during crash periods
spy_dd_s = (spy_prices - spy_prices.cummax()) / spy_prices.cummax()

crash_periods = [
    ('2008 GFC',   '2007-10-01', '2009-03-09'),
    ('2020 COVID', '2020-02-19', '2020-03-23'),
    ('2022 Bear',  '2022-01-03', '2022-10-12'),
]

crash_rows = []
for crash_name, start, end in crash_periods:
    s, e = pd.Timestamp(start), pd.Timestamp(end)
    # SPY B&H
    spy_period = spy_dd_s[(spy_dd_s.index >= s) & (spy_dd_s.index <= e)]
    crash_rows.append({'Crash': crash_name, 'Strategy': 'SPY B&H', 'Max DD %': spy_period.min() * 100})

    # Each leveraged config
    for r in lev_results[1:]:
        dd = r['drawdown']
        period = dd[(dd.index >= s) & (dd.index <= e)]
        if len(period) > 0:
            crash_rows.append({'Crash': crash_name, 'Strategy': r['name'], 'Max DD %': period.min() * 100})

crash_compare = pd.DataFrame(crash_rows).pivot(index='Strategy', columns='Crash', values='Max DD %')

styled = crash_compare.style.format('{:.1f}%').background_gradient(cmap='RdYlGn_r', axis=None)
style_returns_table(styled).set_caption('Drawdown During Crashes: SPY vs Leveraged Deep OTM Puts')

---
## 8. Summary: Sharpe Ratio Comparison

The ultimate risk-adjusted metric. If Spitznagel is right, the leveraged hedged portfolio should have a **higher Sharpe** than SPY alone.

In [ ]:
all_configs = no_lev_results + lev_results[1:] + std_lev_results[1:]

rows = []
for r in all_configs:
    d = r['balance']['% change'].dropna()
    vol = d.std() * np.sqrt(252) * 100
    sharpe = (r['annual_ret'] - 4.0) / vol if vol > 0 else 0
    framing = 'No leverage' if r in no_lev_results else 'Leveraged'
    rows.append({
        'Framing': framing, 'Strategy': r['name'],
        'Annual %': r['annual_ret'], 'Vol %': vol,
        'Max DD %': r['max_dd'], 'Sharpe': sharpe,
        'Excess %': r['excess_annual'],
    })

df_all = pd.DataFrame(rows)
styled = (df_all.style
    .format({'Annual %': '{:.2f}', 'Vol %': '{:.1f}', 'Max DD %': '{:.1f}',
             'Sharpe': '{:.3f}', 'Excess %': '{:+.2f}'})
    .map(color_excess, subset=['Excess %'])
)
style_returns_table(styled).set_caption('Full Comparison: No Leverage (AQR) vs Leverage (Spitznagel)')

---
## 9. Verdict

### The framing matters enormously:

- **AQR framing** (reduce equity to fund puts): puts always lose. This is trivially true because you're replacing a high-return asset with a negative-EV hedge.

- **Spitznagel framing** (keep 100% equity, add puts on top): the answer depends on whether crash payoffs exceed the premium bleed. With real options data, check the tables above.

### What makes or breaks the Spitznagel case:

1. **Deep OTM puts are cheaper per unit of crash exposure** — you get more convexity per dollar
2. **The leverage framing is crucial** — without it, puts can never win
3. **Crash frequency matters** — our 2008–2025 sample has 3 crashes in 17 years
4. **Implementation quality matters** — our monthly rebalancing is crude compared to Universa's active management

### The honest conclusion:

With real options data and the correct leveraged framing, the deep OTM tail hedge has a **fighting chance**. Whether it actually wins depends on the specific parameters and whether crash payoffs in our sample are large enough to overcome the premium bleed.

The numbers above tell the story — check the Sharpe ratios.